## Compulsory Task

In this compulsory task you will clean the country column and parse the date column in the **store_income_data_task.csv** file.

In [ ]:
# Library imports
import pandas as pd
import fuzzywuzzy
from fuzzywuzzy import process
import datetime

# Load up CSV file
income_df = pd.read_csv("store_income_data_task.csv")

In [ ]:
# View the first 5 rows of the dataframe.
income_df.head()

,id,store_name,store_email,department,income,date_measured,country
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,4-2-2006,United States/
1,2,Nordson Corporation,NaN,Tools,$41744177.01,4-1-2006,Britain
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12-9-2003,United States
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,8-5-2006,Britain/
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,21-1-1973,United Kingdom


1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [ ]:
# Display unique country names before cleaning
countries = income_df['country'].unique()
print(f"Number of unique countries before cleaning: {len(countries)}")
countries

# Convert country names to lowercase and remove leading/trailing whitespaces
income_df['country'] = income_df['country'].str.lower().str.strip()

# Replace all null/NaN value country names with 'unknown'
income_df.country.fillna('unknown', inplace=True)

# Display unique country names after cleaning
countries = income_df['country'].unique()
print(f"Number of unique countries after cleaning: {len(countries)}")
countries

Number of unique countries before cleaning: 77
Number of unique countries after cleaning: 37


array(['united states/', 'britain', 'united states', 'britain/',
       'united kingdom', 'u.k.', 'sa', 'u.k/', 'america', 'unknown',
       's.a.', 'england', 'uk', 's.a./', 'u.k', 'america/', 'sa.', '',
       'uk.', 'england/', 'united states of america', 'uk/', 'sa/',
       'england.', 'america.', 's.a..', 'united states of america.',
       'united states of america/', 'united states.',
       's. africasouth africa', 'britain.', '/', 'united kingdom.',
       's. africasouth africa/', 'united kingdom/',
       's. africasouth africa.', '.'], dtype=object)

2.  Clean up the 'country' column so that there are three distinct countries.

     Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [ ]:
# This code cell is used to determine the min_ratio when calling the replace_country_names function
matches = fuzzywuzzy.process.extract("s.a", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches

[('s.a.', 100),
 ('s.a./', 100),
 ('s.a..', 100),
 ('sa', 40),
 ('sa.', 40),
 ('sa/', 40),
 ('u.k.', 33),
 ('u.k/', 33),
 ('u.k', 33),
 ('s. africasouth africa', 26)]

In [ ]:

def replace_country_names(df, column, string_to_match, min_ratio):
    """
    Replaces country names that are a close match using fuzzy logic and a custom ratio

    Arguments:  df - the dataframe to search
                column - the column to search
                string_to_match - the string to search for
                ratio - the minimum ratio to consider a match
    """

    # Get a list of unique names for the specified column
    unique_strings = df[column].unique()

    # Get the top 10 closest matches for the string_to_match value
    matches = fuzzywuzzy.process.extract(string_to_match, unique_strings,
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # Only get matches with a ratio greater than or equal to min_ratio value
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # Get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # Replace all rows with close matches with the input matches
    df.loc[rows_with_matches, column] = string_to_match

    # Let us know when the function is done
    print("Replacement completed")


# The min ratios were determined in the previous code cell by analyzing the relevant
# top 10 closest matches for each string_to_match value
replace_country_names(df=income_df, column='country', string_to_match="uk", min_ratio=40)
replace_country_names(df=income_df, column='country', string_to_match="united kingdom", min_ratio=90)
replace_country_names(df=income_df, column='country', string_to_match="britain", min_ratio=90)
replace_country_names(df=income_df, column='country', string_to_match="england", min_ratio=90)
replace_country_names(df=income_df, column='country', string_to_match="united states", min_ratio=70)
replace_country_names(df=income_df, column='country', string_to_match="america", min_ratio=90)
replace_country_names(df=income_df, column='country', string_to_match="south africa", min_ratio=75)
replace_country_names(df=income_df, column='country', string_to_match="s.a.", min_ratio=40)


Replacement completed
Replacement completed
Replacement completed
Replacement completed
Replacement completed
Replacement completed
Replacement completed
Replacement completed
Name replaced
Name replaced
Name replaced
Name replaced
Name replaced
Name replaced
Name replaced
Name replaced
Name replaced
Name replaced

Number of unique countries: 4


array(['United States', 'United Kingdom', 'South Africa', 'unknown'],
      dtype=object)

In [ ]:
def replace_name(string_to_find, new_name):
    """
    Replaces the specified country name with a specified new name

    Arguments:  string_to_find - country name to be replaced
                new_name - name to replace the old name with
    """
    income_df.replace(string_to_find, new_name, inplace=True)

    # Let us know when the function is done
    print("Name replaced")


replace_name(string_to_find='united states', new_name="United States")
replace_name(string_to_find='america', new_name="United States")
replace_name(string_to_find='s.a.', new_name="South Africa")
replace_name(string_to_find='britain', new_name="United Kingdom")
replace_name(string_to_find='united kingdom', new_name="United Kingdom")
replace_name(string_to_find='uk', new_name="United Kingdom")
replace_name(string_to_find='england', new_name="United Kingdom")
replace_name(string_to_find='/', new_name="unknown")
replace_name(string_to_find='.', new_name="unknown")
replace_name(string_to_find='', new_name="unknown")

# Display unique country names after all cleaning
countries = income_df['country'].unique()
print(f"\nNumber of unique countries: {len(countries)}")
countries

In [ ]:
# Remove the 'unknown' values in  countries to remain with  unique countries
filtered_df = income_df[income_df['country'] != 'unknown']

# Print out the three distinct countries
filtered_df.country.unique()

array(['United States', 'United Kingdom', 'South Africa'], dtype=object)

3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.

In [ ]:
# Convert 'date_measured' from object type to datetime64 type
income_df['date_measured'] = pd.to_datetime(income_df['date_measured'], format='%d-%m-%Y')

# Copy 'days_ago' column from 'date_measured' column and convert to datetime64 type
income_df['days_ago'] = pd.to_datetime(income_df['date_measured'], format='%d-%m-%Y')
income_df['days_ago'] = pd.to_datetime(datetime.date.today(), format='%d-%m-%Y')

# Subtract the date measured from todays date to get the number of days that have passed
income_df['days_ago'] = income_df['days_ago'] - income_df['date_measured']
income_df.head()

In [ ]:
# Display the first five observation of the days ago column.
income_df['days_ago'].head()